## Robustness of Numerai Models 

1. Validation/Test Period Performance
    - Validation Performance


2. Standard Error of Models 
    - Calculate Performance of Models for each indiviaul seed 
    
    
3. Skipping intial Trees
    - For LightGBM-gbdt models, report results 
    
    
4. Random Feature Neutralisation 
    - For a given set of example models (LightGBM-gbdt) 
    - Perform Random Feature Neutralisaion for 200 times 
    - Record the correlations 



In [1]:
import numpy as np
import pandas as pd
import os, json, datetime, sys, joblib, glob
import lightgbm
import torch
from pytrend.numerai import (
    create_era_index,
    load_numerai_data,
    predict_numerai_multiple,
    dynamic_feature_neutralisation,
    run_numerai_models_performances,
    score_numerai,
)
from pytrend.util import dynamic_model_selection_masks, walk_forward_dynamic_models, strategy_metrics
from pytrend.benchmark import benchmark_pipeline
from pytrend.optimisation import numerai_optimisation_pipeline_optuna

## Debug

In [2]:
from pytrend.feature import NumeraiTransformer
import joblib
from pytrend.benchmark import load_best_model

In [3]:
for method in ['lightgbm-gbdt','lightgbm-dart','Numerai-MLP',]:
    for seed in range(2000,2100,100):
        parametername = f'../numerai_models_v4_thesis/{method}_None_1_{seed}.parameters' 
        if os.path.exists(parametername):
            modelname = parametername.replace('.parameters','.model')
            parameters = joblib.load(parametername)
            print(seed, parameters['parameters']['model']['tabular_model'])
            print(seed, parameters['parameters']['model']['target_columns'])

            feature_eng_parameters = parameters["parameters"]["feature_eng"]
            features_raw = pd.read_parquet('../data/v4_live_int8.parquet')
            selected_cols = parameters["parameters"]["model"]["feature_columns"]
            if parameters['parameters']['model']["feature_engineering"]:
                transformer = NumeraiTransformer(**feature_eng_parameters)
                transformer.data = parameters["transformer"]
                features = transformer.transform(
                    features_raw[selected_cols], is_train=False
                )
            else:
                features = features_raw[selected_cols]
            trained_model = load_best_model(
            parameters["parameters"]["model"]["tabular_model"], modelname
            )
            predictions_raw = trained_model.predict(
            features.values,
            )
            print(len(selected_cols),predictions_raw.shape,predictions_raw.mean(),predictions_raw.std(),predictions_raw.max(),predictions_raw.min())

2000 lightgbm-gbdt
2000 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999,) 0.008978565053280211 0.01004719615030952 0.04232926892571976 -0.025437110192893343
2000 lightgbm-dart
2000 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999,) 0.01000229823361673 0.015329623199640619 0.06785093645414622 -0.04456871616625498
2000 Numerai-MLP
2000 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999, 1) -0.0027154086 0.009404434 0.08224736 -0.049243275


In [4]:
for method in ['lightgbm-gbdt','lightgbm-dart','Numerai-MLP',]:
    for seed in range(0,100,100):
        parametername = f'../numerai_models_v4_thesis/{method}_numerai_1_{seed}.parameters' 
        print(parametername)
        if os.path.exists(parametername):
            modelname = parametername.replace('.parameters','.model')
            parameters = joblib.load(parametername)
            print(seed, parameters['parameters']['model']['tabular_model'])
            print(seed, parameters['parameters']['model']['target_columns'])

            feature_eng_parameters = parameters["parameters"]["feature_eng"]
            features_raw = pd.read_parquet('../data/v4_live_int8.parquet')
            selected_cols = parameters["parameters"]["model"]["feature_columns"]
            if parameters['parameters']['model']["feature_engineering"]:
                transformer = NumeraiTransformer(**feature_eng_parameters)
                transformer.data = parameters["transformer"]
                features = transformer.transform(
                    features_raw[selected_cols], is_train=False
                )
            else:
                features = features_raw[selected_cols]
            trained_model = load_best_model(
            parameters["parameters"]["model"]["tabular_model"], modelname
            )
            predictions_raw = trained_model.predict(
            features.values,
            )
            print(len(selected_cols),predictions_raw.shape,predictions_raw.mean(),predictions_raw.std(),predictions_raw.max(),predictions_raw.min())

../numerai_models_v4_thesis/lightgbm-gbdt_numerai_1_0.parameters
0 lightgbm-gbdt
0 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999,) 0.006127853743345332 0.008580619571471215 0.03372439425233676 -0.022632588920557254
../numerai_models_v4_thesis/lightgbm-dart_numerai_1_0.parameters
0 lightgbm-dart
0 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999,) 0.005224143453633915 0.009220809843572292 0.03534838416450697 -0.02936440938380566
../numerai_models_v4_thesis/Numerai-MLP_numerai_1_0.parameters
0 Numerai-MLP
0 Index(['target_nomi_v4_20'], dtype='object')
1181 (4999, 1) -0.009440164 0.0062019154 0.028122297 -0.018086046


### Validation period Performances

In [2]:
numerai_files = {
    "dataset": "../data/v4_all_int8.parquet",
    "feature_corr": "../data/v4_feature_corr.parquet",
    "feature_metadata": "../data/v4_features.json",
}
MODEL_FOLDER = "../numerai_models_v4_thesis"
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_robustness"
### Get Model Performances in validation period
feature_corr = pd.read_parquet(numerai_files["feature_corr"])
with open(numerai_files["feature_metadata"], "r") as f:
    feature_metadata = json.load(f)
features_optimizer = feature_metadata["feature_sets"]["fncv3_features"]

In [3]:
def benchmark_validation_performance(Numerai_Model_Names, startera="0521", endera="0621", gbm_start_iteration=0, ):
    strategy_flavour, strategy_correlations, predictions = save_model_performance_test(Numerai_Model_Names,feature_corr,features_optimizer,
                                startera=startera,endera=endera,data_file=numerai_files['dataset'], gbm_start_iteration=gbm_start_iteration)
    parametername = Numerai_Model_Names[0]
    no_models = len(Numerai_Model_Names)
    stem = parametername.split("/")[-1].replace(".parameters", "")
    correlations_filename = f"{PERFORMANCES_FOLDER}/{stem}_{no_models}.csv"
    strategy_correlations["neutralised_correlation"].to_csv(correlations_filename)
    return None

In [4]:
from pytrend.numerai import save_model_performance_test

In [5]:
startera="0521"
endera="0620"
gbm_start_iteration=0

for ML_METHOD in ['Numerai-MLP', ]:

    FEATURE_ENG = "None"
    Numerai_Model_Names = [f"{MODEL_FOLDER}/{ML_METHOD}_{FEATURE_ENG}_1_{seed}.parameters" for seed in range(3000,3010)]

    strategy_flavour, strategy_correlations, predictions = save_model_performance_test(Numerai_Model_Names,feature_corr,features_optimizer,
                            startera=startera,endera=endera,data_file=numerai_files['dataset'], data_version="v4-all",gbm_start_iteration=gbm_start_iteration)
    parametername = Numerai_Model_Names[0]
    no_models = len(Numerai_Model_Names)
    stem = parametername.split("/")[-1].replace(".parameters", "")
    correlations_filename = f"{PERFORMANCES_FOLDER}/{stem}_{no_models}_validate.csv"
    strategy_correlations['neutralised_correlation'].to_csv(correlations_filename)

In [6]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_robustness"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*validate.csv"))

In [7]:
validate_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    validate_performances.append(df[cols])
    
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
validate_performances.append(factor_linear)

In [8]:
pd.DataFrame(pd.concat(validate_performances,axis=1).loc['2012-12-21':'2014-11-14'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_2000_10-baseline,0.0443,0.0201,-0.1135,-0.0930,0.0065,2.2058,6.8154
Numerai-MLP_None_1_3000_10-baseline,0.0431,0.0189,-0.0901,-0.1193,0.0012,2.2785,35.9167
Numerai-MLP_numerai_1_0_10-baseline,0.0423,0.0208,-0.3358,0.3518,0.0241,2.0338,1.7552
lightgbm-dart_None_1_2000_10-baseline,0.0475,0.0199,-0.1854,0.2465,0.0079,2.3883,6.0127
lightgbm-gbdt_None_1_2000_10-baseline,0.0500,0.0224,-0.4144,1.6728,0.0235,2.2335,2.1277
lightgbm-gbdt_numerai_1_0_10-baseline,0.0483,0.0229,-0.3655,1.4800,0.0307,2.1144,1.5733
lightgbm-goss_None_1_2000_10-baseline,0.0302,0.0234,-0.2238,0.8953,0.0877,1.2877,0.3444
lightgbm-goss_numerai_1_0_10-baseline,0.0288,0.0219,-0.2434,0.9373,0.0687,1.3136,0.4192
tabnet_None_1_2000_10-baseline,0.0362,0.0189,-0.6221,-0.1262,0.0199,1.9125,1.8191
factor_momentum,0.0229,0.0170,-0.6429,0.8299,0.0691,1.3495,0.3314


### Test Period Performances 

In [9]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_thesis"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*1_2000_10*.csv")) + sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*1_0_10*.csv")) 

In [10]:
test_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    test_performances.append(df[cols])
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
test_performances.append(factor_linear)

In [11]:
pd.DataFrame(pd.concat(test_performances,axis=1).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_2000_10-baseline,0.0258,0.0289,0.1588,-0.1685,0.1668,0.8931,0.1547
lightgbm-dart_None_1_2000_10-baseline,0.0278,0.0284,-0.0012,0.0398,0.1622,0.9791,0.1714
lightgbm-gbdt_None_1_2000_10-baseline,0.0262,0.0321,0.0033,0.0714,0.2378,0.8140,0.1102
lightgbm-goss_None_1_2000_10-baseline,0.0156,0.0318,0.2734,0.1345,0.7528,0.4896,0.0207
tabnet_None_1_2000_10-baseline,0.0161,0.0296,0.1490,0.0642,0.5811,0.5431,0.0277
Numerai-MLP_numerai_1_0_10-baseline,0.0237,0.0330,-0.0712,0.0102,0.2912,0.7189,0.0814
lightgbm-dart_numerai_1_0_10-baseline,0.0265,0.0319,0.0179,0.0863,0.2151,0.8313,0.1232
lightgbm-gbdt_numerai_1_0_10-baseline,0.0253,0.0327,-0.0075,0.1318,0.3064,0.7731,0.0826
lightgbm-goss_numerai_1_0_10-baseline,0.0169,0.0297,0.2283,0.4959,0.5539,0.5695,0.0305
factor_momentum,0.0080,0.0275,0.0622,0.0680,0.7877,0.2923,0.0102


In [12]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_thesis"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*Numerai-MLP*1_3000_10*.csv")) + sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*Numerai-MLP*1_1000_10*.csv")) 

In [13]:
test_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    test_performances.append(df[cols])
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
test_performances.append(factor_linear)

In [14]:
pd.DataFrame(pd.concat(test_performances,axis=1).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_3000_10-baseline,0.0259,0.0299,0.1861,-0.2242,0.1777,0.8642,0.1458
factor_momentum,0.0080,0.0275,0.0622,0.0680,0.7877,0.2923,0.0102


### Test Period Performances in different CV 

In [15]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_thesis"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*1_*10_10*.csv"))

In [16]:
test_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    test_performances.append(df[cols])
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
test_performances.append(factor_linear)

In [17]:
pd.DataFrame(pd.concat(test_performances,axis=1).loc['2017-04-14':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_2010_10-baseline,0.0215,0.0289,0.1116,0.1299,0.2307,0.7446,0.0932
Numerai-MLP_None_1_3010_10-baseline,0.0214,0.0300,-0.0293,0.0536,0.2781,0.7139,0.0770
lightgbm-dart_None_1_2010_10-baseline,0.0250,0.0278,0.2595,0.2396,0.1817,0.8990,0.1376
lightgbm-dart_numerai_1_10_10-baseline,0.0233,0.0315,0.2101,0.1975,0.2811,0.7377,0.0829
lightgbm-gbdt_None_1_2010_10-baseline,0.0231,0.0324,0.1933,0.2101,0.3227,0.7104,0.0716
lightgbm-gbdt_numerai_1_10_10-baseline,0.0216,0.0335,0.1578,0.2192,0.3810,0.6465,0.0567
lightgbm-goss_None_1_2010_10-baseline,0.0128,0.0314,0.2275,0.1455,0.7318,0.4075,0.0175
tabnet_None_1_2010_10-baseline,0.0152,0.0301,0.2051,-0.0042,0.4803,0.5037,0.0316
factor_momentum,0.0084,0.0282,0.0614,-0.0974,0.7877,0.2977,0.0107


In [18]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_thesis"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*1_*20_10*.csv"))

In [19]:
test_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    test_performances.append(df[cols])
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
test_performances.append(factor_linear)

In [20]:
pd.DataFrame(pd.concat(test_performances,axis=1).loc['2019-03-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_2020_10-baseline,0.0207,0.0223,0.1678,0.3054,0.1171,0.9270,0.1768
Numerai-MLP_None_1_3020_10-baseline,0.0189,0.0218,0.1232,0.6243,0.1777,0.8684,0.1064
lightgbm-dart_None_1_2020_10-baseline,0.0247,0.0279,0.2022,0.2128,0.1380,0.8834,0.1790
lightgbm-dart_numerai_1_20_10-baseline,0.0257,0.0314,0.2246,0.2022,0.1283,0.8193,0.2003
lightgbm-gbdt_None_1_2020_10-baseline,0.0250,0.0324,0.1326,0.0730,0.1584,0.7729,0.1578
lightgbm-gbdt_numerai_1_20_10-baseline,0.0247,0.0329,0.0846,-0.0985,0.1732,0.7515,0.1426
lightgbm-goss_None_1_2020_10-baseline,0.0189,0.0282,0.1450,-0.1115,0.2256,0.6719,0.0838
tabnet_None_1_2020_10-baseline,0.0186,0.0268,0.1784,0.0339,0.1896,0.6943,0.0981
factor_momentum,0.0110,0.0302,-0.0953,-0.2391,0.7869,0.3655,0.0140


In [21]:
import glob
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_thesis"
files = sorted(glob.glob(f"{PERFORMANCES_FOLDER}/*1_*30_10*.csv"))

In [22]:
test_performances = list()
for f in files:
    df = pd.read_csv(f,index_col=0,)
    df.index = pd.to_datetime(df.index)
    cols = [x for x in df.columns if x.endswith('baseline')]
    test_performances.append(df[cols])
factor_linear = create_era_index(pd.read_csv("../data/v4_factor_portfolio.csv",index_col=0))
test_performances.append(factor_linear)

In [23]:
pd.DataFrame(pd.concat(test_performances,axis=1).loc['2021-02-12':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
Numerai-MLP_None_1_2030_10-baseline,0.0290,0.0306,0.0754,0.3384,0.1575,0.9465,0.1841
Numerai-MLP_None_1_3030_10-baseline,0.0319,0.0284,0.1488,0.2872,0.1009,1.1204,0.3162
lightgbm-dart_None_1_2030_10-baseline,0.0318,0.0295,0.0462,0.9992,0.1557,1.0768,0.2042
lightgbm-dart_numerai_1_30_10-baseline,0.0341,0.0329,0.0606,0.8977,0.1488,1.0375,0.2292
lightgbm-gbdt_None_1_2030_10-baseline,0.0294,0.0313,0.0904,0.1631,0.0774,0.9403,0.3798
lightgbm-gbdt_numerai_1_30_10-baseline,0.0298,0.0311,0.1446,0.0688,0.0668,0.9563,0.4461
lightgbm-goss_None_1_2030_10-baseline,0.0238,0.0281,0.1297,-0.2626,0.0847,0.8488,0.2810
tabnet_None_1_2030_10-baseline,0.0194,0.0230,-0.0297,0.2802,0.1459,0.8427,0.1330
factor_momentum,0.0210,0.0261,0.3316,0.0921,0.0712,0.8035,0.2949


### Regime Analysis 

In [24]:
def regime_analysis(df,performance_col='correlation',regime_columns='regime',):
    ans = df.groupby(regime_columns).agg({performance_col:strategy_metrics})
    ans_df = pd.DataFrame(ans[performance_col].values.tolist())
    ans_df.index = ans.index
    return ans_df.reset_index()

In [25]:
def create_leaderboard(performances_folder, models_subset="Standard", lookback=52, searchkey='*'):
    performances_files = sorted(glob.glob(f"{performances_folder}/{searchkey}.csv"))
    models_list = list()
    for f in performances_files:
        model_no = int(f.split(".csv")[0].split("_")[-2])
        model_seq = int(f.split(".csv")[0].split("_")[-1])
        model_name = "_".join(f.split(".csv")[0].split("/")[-1].split("_")[:3])
        if os.path.isfile(f):
            df = pd.read_csv(f, index_col=0).sort_index()
            df = df[~df.index.duplicated()]
            df.index = pd.to_datetime(df.index)
            models_list.append(df)

    ### Construct Leaderboard of all trained models based on last 52 weeks performances
    dynamic_models_collection = walk_forward_dynamic_models(models_list)
    leaderboard = pd.DataFrame(
        dynamic_models_collection[models_subset]
        .sort_index()
        .iloc[-1 * lookback :]
        .apply(strategy_metrics)
        .to_dict()
    ).transpose()
    leaderboard.index = dynamic_models_collection[models_subset].columns
    leaderboard["flavour"] = [x[-1] for x in leaderboard.index.str.split("-")]
    leaderboard["model_name"] = [
        "-".join(x[:-1]) for x in leaderboard.index.str.split("-")
    ]
    leaderboard["model_seq"] = [x[-1] for x in leaderboard["model_name"].str.split("_")]
    leaderboard["model_seed"] = [
        x[-2] for x in leaderboard["model_name"].str.split("_")
    ]
    leaderboard["model_stem"] = [
        "_".join("-".join(x[:-1]).split("_")[:-2])
        for x in leaderboard.index.str.split("-")
    ]
    return leaderboard, dynamic_models_collection

In [26]:
from pytrend.numerai import create_era_index
from pytrend.numerai import convert_era_to_datetime

In [27]:
df = pd.read_csv('../data/v4_factor_portfolio.csv',index_col=0)
factor_portfolios = create_era_index(df[['factor_momentum']].rolling(20).mean().shift(6))
NRVIX = create_era_index(df[['factor_momentum']].rolling(52).std()).shift(6).dropna()
NRVIX.columns = ['NRVIX']

In [28]:
def model_performances_by_regime(dynamic_models_collection,model_set = 'Optimizer',):
    results = list()
    for col in dynamic_models_collection[model_set].columns:
        if 'baseline' in col or 'optimizer' in col or 'mean' in col or 'volatility' in col:
            sample_data = dynamic_models_collection[model_set][col].loc['2015-05-15':'2022-09-23']
            sample_performance = pd.DataFrame(sample_data)
            sample_performance.columns = ['correlation']
            sample_performance['factor_momentum'] =  factor_portfolios['factor_momentum']
            sample_performance['NRVIX'] =  NRVIX['NRVIX']
            sample_performance['regime'] = np.where(sample_performance['NRVIX']>0.025, 'high_vol', 'low_vol')
            temp = regime_analysis(sample_performance)
            temp['model_name'] = col
            results.append(temp)

            sample_performance['regime'] = np.where(sample_performance['NRVIX']>=0, 'all', 'dummy')
            temp = regime_analysis(sample_performance)
            temp['model_name'] = col
            results.append(temp) 
    return pd.concat(results)

In [29]:
leaderboard, dynamic_models_collection = create_leaderboard('../numerai_models_performances_v4_thesis',searchkey='*None_1_2000*')
model_performances_by_regime(dynamic_models_collection,'Optimizer')

,regime,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,model_name
0,high_vol,0.0196,0.0193,0.4577,-0.1314,0.0326,1.0191,0.6012,Numerai-MLP_None_1_2000_10-optimizer
1,low_vol,0.0165,0.0210,0.0394,0.1674,0.2606,0.7875,0.0633,Numerai-MLP_None_1_2000_10-optimizer
0,all,0.0179,0.0203,0.1723,0.1203,0.2606,0.8798,0.0687,Numerai-MLP_None_1_2000_10-optimizer
0,high_vol,0.0227,0.0163,0.1300,-0.4480,0.0223,1.3888,1.0179,lightgbm-dart_None_1_2000_10-optimizer
1,low_vol,0.0206,0.0195,-0.1499,0.1267,0.1153,1.0576,0.1787,lightgbm-dart_None_1_2000_10-optimizer
0,all,0.0215,0.0182,-0.0933,0.0753,0.1153,1.1806,0.1865,lightgbm-dart_None_1_2000_10-optimizer
0,high_vol,0.0217,0.0198,0.2661,0.3629,0.0364,1.0953,0.5962,lightgbm-gbdt_None_1_2000_10-optimizer
1,low_vol,0.0194,0.0220,0.0020,0.0572,0.1998,0.8820,0.0971,lightgbm-gbdt_None_1_2000_10-optimizer
0,all,0.0204,0.0211,0.0781,0.1959,0.1998,0.9665,0.1021,lightgbm-gbdt_None_1_2000_10-optimizer
0,high_vol,0.0090,0.0166,-0.0305,-0.0939,0.1050,0.5397,0.0857,lightgbm-goss_None_1_2000_10-optimizer


In [30]:
leaderboard, dynamic_models_collection = create_leaderboard('../numerai_models_performances_v4_thesis',searchkey='*None_1_2000*')
model_performances_by_regime(dynamic_models_collection,'Baseline')

,regime,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,model_name
0,high_vol,0.0298,0.0276,0.1759,-0.4294,0.1247,1.0802,0.2390,Numerai-MLP_None_1_2000_10-baseline
1,low_vol,0.0228,0.0296,0.1940,-0.0072,0.1668,0.7721,0.1367,Numerai-MLP_None_1_2000_10-baseline
0,all,0.0258,0.0289,0.1588,-0.1685,0.1668,0.8931,0.1547,Numerai-MLP_None_1_2000_10-baseline
0,high_vol,0.0314,0.0251,0.0470,0.0206,0.0657,1.2510,0.4779,lightgbm-dart_None_1_2000_10-baseline
1,low_vol,0.0252,0.0305,0.0639,-0.0515,0.1622,0.8257,0.1554,lightgbm-dart_None_1_2000_10-baseline
0,all,0.0278,0.0284,-0.0012,0.0398,0.1622,0.9791,0.1714,lightgbm-dart_None_1_2000_10-baseline
0,high_vol,0.0308,0.0293,0.1025,0.3008,0.1123,1.0497,0.2743,lightgbm-gbdt_None_1_2000_10-baseline
1,low_vol,0.0227,0.0338,0.0307,-0.0917,0.2378,0.6727,0.0955,lightgbm-gbdt_None_1_2000_10-baseline
0,all,0.0262,0.0321,0.0033,0.0714,0.2378,0.8140,0.1102,lightgbm-gbdt_None_1_2000_10-baseline
0,high_vol,0.0201,0.0289,0.2002,0.0214,0.1334,0.6947,0.1507,lightgbm-goss_None_1_2000_10-baseline


### Dynamic Feature Neutralisation

In [31]:
pd.set_option('display.max_rows', 200)

In [32]:
leaderboard, dynamic_models_collection = create_leaderboard('../numerai_models_performances_v4_thesis',searchkey='*None_1_2000*')
model_performances_by_regime(dynamic_models_collection,'Standard')

,regime,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,model_name
0,high_vol,0.0208,0.0188,0.2818,-0.1392,0.0430,1.1057,0.4837,Numerai-MLP_None_1_2000_10-high_mean_standard
1,low_vol,0.0170,0.0210,-0.1110,-0.1149,0.1283,0.8118,0.1325,Numerai-MLP_None_1_2000_10-high_mean_standard
0,all,0.0186,0.0201,-0.0043,-0.0370,0.1283,0.9256,0.1450,Numerai-MLP_None_1_2000_10-high_mean_standard
0,high_vol,0.0187,0.0165,0.0673,-0.1407,0.0336,1.1368,0.5565,Numerai-MLP_None_1_2000_10-high_volatility_sta...
1,low_vol,0.0194,0.0177,-0.2667,-0.0416,0.0730,1.0990,0.2658,Numerai-MLP_None_1_2000_10-high_volatility_sta...
0,all,0.0191,0.0172,-0.1364,-0.0909,0.0730,1.1150,0.2616,Numerai-MLP_None_1_2000_10-high_volatility_sta...
0,high_vol,0.0205,0.0183,-0.0126,-0.1174,0.0806,1.1212,0.2543,Numerai-MLP_None_1_2000_10-low_mean_standard
1,low_vol,0.0215,0.0187,0.0778,0.0191,0.0496,1.1496,0.4335,Numerai-MLP_None_1_2000_10-low_mean_standard
0,all,0.0211,0.0185,0.0417,-0.0457,0.0806,1.1387,0.2618,Numerai-MLP_None_1_2000_10-low_mean_standard
0,high_vol,0.0222,0.0194,0.4856,0.1595,0.0397,1.1442,0.5592,Numerai-MLP_None_1_2000_10-low_volatility_stan...


#### Correlation Structure 

In [13]:
def correlation_structure(model_name):
    leaderboard, dynamic_models_collection = create_leaderboard('../numerai_models_performances_v4_thesis',searchkey=f'{model_name}*')
    selected_columns =  [f'{model_name}-{x}_standard' for x in ['high_mean','low_mean','high_volatility','low_volatility']] 
    df = pd.concat([dynamic_models_collection['Standard'][selected_columns],dynamic_models_collection['Ensemble']],axis=1)
    return df.loc['2015-05-15':'2022-09-23'].corr().describe()    
   

In [14]:
model_name = 'lightgbm-dart_None_1_2000_10'
correlation_structure(model_name)

,lightgbm-dart_None_1_2000_10-high_mean_standard,lightgbm-dart_None_1_2000_10-low_mean_standard,lightgbm-dart_None_1_2000_10-high_volatility_standard,lightgbm-dart_None_1_2000_10-low_volatility_standard,lightgbm-dart_None_1_2000_10-baseline,lightgbm-dart_None_1_2000_10-high_autocorrelation_standard,lightgbm-dart_None_1_2000_10-high_calmar_standard,lightgbm-dart_None_1_2000_10-high_drawdown_standard,lightgbm-dart_None_1_2000_10-high_kurt_standard,lightgbm-dart_None_1_2000_10-high_mean_standard,...,lightgbm-dart_None_1_2000_10-high_volatility_standard,lightgbm-dart_None_1_2000_10-low_autocorrelation_standard,lightgbm-dart_None_1_2000_10-low_calmar_standard,lightgbm-dart_None_1_2000_10-low_drawdown_standard,lightgbm-dart_None_1_2000_10-low_kurt_standard,lightgbm-dart_None_1_2000_10-low_mean_standard,lightgbm-dart_None_1_2000_10-low_sharpe_standard,lightgbm-dart_None_1_2000_10-low_skew_standard,lightgbm-dart_None_1_2000_10-low_volatility_standard,lightgbm-dart_None_1_2000_10-optimizer
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.807713,0.810140,0.805699,0.780013,0.745264,0.808812,0.800293,0.789049,0.801011,0.807713,...,0.805699,0.800745,0.801369,0.807145,0.824536,0.810140,0.813087,0.811791,0.780013,0.735226
std,0.095080,0.095264,0.078686,0.084140,0.077660,0.054034,0.086613,0.061700,0.065230,0.095080,...,0.078686,0.057496,0.085413,0.056011,0.050910,0.095264,0.088658,0.056720,0.084140,0.072493
min,0.689255,0.696522,0.683379,0.646250,0.639888,0.725166,0.696522,0.673371,0.647831,0.689255,...,0.683379,0.704777,0.689255,0.703304,0.708508,0.696522,0.702945,0.704178,0.646250,0.639888
25%,0.735767,0.740220,0.783052,0.742402,0.704328,0.775051,0.745805,0.754358,0.773590,0.735767,...,0.783052,0.774346,0.775677,0.775055,0.800005,0.740220,0.755584,0.784842,0.742402,0.715060
50%,0.800390,0.807973,0.809409,0.766467,0.725328,0.802946,0.780787,0.785422,0.804486,0.800390,...,0.809409,0.804061,0.788409,0.808426,0.827516,0.807973,0.802650,0.812743,0.766467,0.734500
75%,0.821915,0.838686,0.822018,0.794494,0.772529,0.830985,0.821254,0.810295,0.824461,0.821915,...,0.822018,0.815564,0.812409,0.823245,0.833520,0.838686,0.829865,0.826985,0.794494,0.741259
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
model_name = 'lightgbm-gbdt_None_1_2000_10'
correlation_structure(model_name)

,lightgbm-gbdt_None_1_2000_10-high_mean_standard,lightgbm-gbdt_None_1_2000_10-low_mean_standard,lightgbm-gbdt_None_1_2000_10-high_volatility_standard,lightgbm-gbdt_None_1_2000_10-low_volatility_standard,lightgbm-gbdt_None_1_2000_10-baseline,lightgbm-gbdt_None_1_2000_10-high_autocorrelation_standard,lightgbm-gbdt_None_1_2000_10-high_calmar_standard,lightgbm-gbdt_None_1_2000_10-high_drawdown_standard,lightgbm-gbdt_None_1_2000_10-high_kurt_standard,lightgbm-gbdt_None_1_2000_10-high_mean_standard,...,lightgbm-gbdt_None_1_2000_10-high_volatility_standard,lightgbm-gbdt_None_1_2000_10-low_autocorrelation_standard,lightgbm-gbdt_None_1_2000_10-low_calmar_standard,lightgbm-gbdt_None_1_2000_10-low_drawdown_standard,lightgbm-gbdt_None_1_2000_10-low_kurt_standard,lightgbm-gbdt_None_1_2000_10-low_mean_standard,lightgbm-gbdt_None_1_2000_10-low_sharpe_standard,lightgbm-gbdt_None_1_2000_10-low_skew_standard,lightgbm-gbdt_None_1_2000_10-low_volatility_standard,lightgbm-gbdt_None_1_2000_10-optimizer
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.790077,0.787929,0.782019,0.754845,0.719422,0.790057,0.780070,0.768039,0.773363,0.790077,...,0.782019,0.778639,0.775511,0.781769,0.797988,0.787929,0.789680,0.788948,0.754845,0.698644
std,0.104366,0.105660,0.087909,0.095856,0.092168,0.058956,0.094882,0.070542,0.071563,0.104366,...,0.087909,0.063435,0.095661,0.062747,0.061364,0.105660,0.099462,0.063499,0.095856,0.088604
min,0.654571,0.665720,0.644452,0.583338,0.563177,0.701145,0.666108,0.603894,0.604606,0.654571,...,0.644452,0.684502,0.654571,0.673620,0.653168,0.665720,0.672518,0.675749,0.583338,0.563177
25%,0.714130,0.708411,0.762423,0.714566,0.658813,0.755797,0.721364,0.752955,0.733520,0.714130,...,0.762423,0.743756,0.733100,0.750015,0.780949,0.708411,0.731066,0.761105,0.714566,0.682440
50%,0.783299,0.777763,0.778477,0.743999,0.704739,0.788785,0.761867,0.768346,0.777049,0.783299,...,0.778477,0.776611,0.761584,0.775214,0.801573,0.777763,0.769222,0.799373,0.743999,0.691698
75%,0.804679,0.815788,0.802550,0.767732,0.757225,0.810158,0.795101,0.777679,0.802369,0.804679,...,0.802550,0.795061,0.786122,0.799409,0.812027,0.815788,0.807618,0.808999,0.767732,0.708493
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
model_name = 'Numerai-MLP_None_1_2000_10'
correlation_structure(model_name)

,Numerai-MLP_None_1_2000_10-high_mean_standard,Numerai-MLP_None_1_2000_10-low_mean_standard,Numerai-MLP_None_1_2000_10-high_volatility_standard,Numerai-MLP_None_1_2000_10-low_volatility_standard,Numerai-MLP_None_1_2000_10-baseline,Numerai-MLP_None_1_2000_10-high_autocorrelation_standard,Numerai-MLP_None_1_2000_10-high_calmar_standard,Numerai-MLP_None_1_2000_10-high_drawdown_standard,Numerai-MLP_None_1_2000_10-high_kurt_standard,Numerai-MLP_None_1_2000_10-high_mean_standard,...,Numerai-MLP_None_1_2000_10-high_volatility_standard,Numerai-MLP_None_1_2000_10-low_autocorrelation_standard,Numerai-MLP_None_1_2000_10-low_calmar_standard,Numerai-MLP_None_1_2000_10-low_drawdown_standard,Numerai-MLP_None_1_2000_10-low_kurt_standard,Numerai-MLP_None_1_2000_10-low_mean_standard,Numerai-MLP_None_1_2000_10-low_sharpe_standard,Numerai-MLP_None_1_2000_10-low_skew_standard,Numerai-MLP_None_1_2000_10-low_volatility_standard,Numerai-MLP_None_1_2000_10-optimizer
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.823534,0.820821,0.799717,0.806779,0.752899,0.818071,0.823479,0.780244,0.808918,0.823534,...,0.799717,0.817991,0.822441,0.820992,0.830016,0.820821,0.827441,0.822895,0.806779,0.751464
std,0.085639,0.086764,0.080827,0.077625,0.078462,0.050122,0.077344,0.060704,0.056173,0.085639,...,0.080827,0.054656,0.073323,0.058111,0.050596,0.086764,0.080624,0.052120,0.077625,0.069936
min,0.725053,0.724320,0.670862,0.678246,0.626523,0.743214,0.732506,0.666429,0.672072,0.725053,...,0.670862,0.705218,0.737546,0.695971,0.722272,0.724320,0.735998,0.725527,0.678246,0.626523
25%,0.765240,0.758123,0.785717,0.782124,0.709482,0.796612,0.765505,0.753084,0.781025,0.765240,...,0.785717,0.800977,0.775224,0.793094,0.809237,0.758123,0.766211,0.814238,0.782124,0.718130
50%,0.804973,0.808479,0.806007,0.802763,0.745179,0.819996,0.808091,0.777298,0.805957,0.804973,...,0.806007,0.814339,0.814146,0.819244,0.825841,0.808479,0.813122,0.819230,0.802763,0.745203
75%,0.843787,0.835435,0.812644,0.823801,0.779440,0.833917,0.840693,0.797556,0.824478,0.843787,...,0.812644,0.838175,0.832502,0.841986,0.847809,0.835435,0.838931,0.839786,0.823801,0.756710
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Select the Dynamic Feature Neutralisation method

In [17]:
def switching_dfn(df,no_tops_list=[1,],random_mode=False,):
    df.index = pd.to_datetime(df.index)
    if not random_mode:
        flavours = ['optimizer','low_volatility_standard','high_volatility_standard','low_mean_standard','high_mean_standard',]
        selected_columns = [f'{model_name}-{flavour}' for flavour in flavours ]
        data = df[selected_columns].loc['2015-05-15':'2022-09-23']
    else:
        data = df.loc['2015-05-15':'2022-09-23']
        
    recent_results = list()
    portfolios = dict()
    gap = 6
    lookback = 52
    report_days = 5200
    criteria = ['mean', 'sharpe', 'calmar', ]
    for no_tops in no_tops_list:
        dynamic_masks = dynamic_model_selection_masks(data,top_models=no_tops,lookback=lookback,gap=gap)
        for base_method in criteria:
            for method in [f'{base_method}_max', ]:
                portfolio = (dynamic_masks[method]*data).sum(axis=1,min_count=1)
                portfolio.fillna(data.mean(axis=1),inplace=True)
                portfolios[f"{method}_{no_tops}_lookback_{lookback}"] = portfolio.tail(report_days)
                performances = strategy_metrics(portfolio.tail(report_days))
                performances['method'] = method
                performances['no_tops'] = no_tops   
                performances['lookback'] = lookback
                recent_results.append(performances)          


    #portfolio = data.sort_index().mean(axis=1).iloc[lookback+gap:]
    portfolio = data.sort_index().mean(axis=1)
    performances = strategy_metrics(portfolio.tail(report_days))  
    performances['method'] = 'average'
    performances['no_tops'] = 0   
    performances['lookback'] = lookback
    recent_results.append(performances)
    portfolios[f"average_0"] = portfolio.tail(report_days) 
    return pd.DataFrame(recent_results)

In [18]:
model_name = 'lightgbm-dart_None_1_2000_10'
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0246,0.0180,0.0996,-0.3206,0.0533,1.3654,0.4615,mean_max,1,52
1,0.0234,0.0165,0.0268,-0.3042,0.0533,1.4148,0.4390,sharpe_max,1,52
2,0.0225,0.0171,0.0291,-0.3466,0.0350,1.3122,0.6429,calmar_max,1,52
3,0.0229,0.0160,-0.0200,-0.4117,0.0619,1.4323,0.3700,average,0,52


In [19]:
model_name = 'lightgbm-gbdt_None_1_2000_10'
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0228,0.0201,0.1367,0.0667,0.0729,1.1342,0.3128,mean_max,1,52
1,0.0224,0.0187,0.0332,-0.0528,0.0729,1.1966,0.3073,sharpe_max,1,52
2,0.0216,0.0195,0.1353,-0.0081,0.0508,1.1102,0.4252,calmar_max,1,52
3,0.0216,0.0177,0.0747,-0.3600,0.0710,1.2165,0.3042,average,0,52


In [20]:
## MLP 
model_name = 'Numerai-MLP_None_1_2000_10'
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0212,0.0191,0.3312,0.0059,0.0878,1.1124,0.2415,mean_max,1,52
1,0.0207,0.0186,0.0690,-0.0971,0.0878,1.1110,0.2358,sharpe_max,1,52
2,0.0187,0.0201,0.0416,0.2272,0.1973,0.9309,0.0948,calmar_max,1,52
3,0.0195,0.0175,0.1543,-0.1751,0.0918,1.1149,0.2124,average,0,52


### Standard Error of models 

In [21]:
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_robustness/single"

In [22]:
def standard_error_models(model_name):
    single_performances = dict()
    for model_no in range(2000,2010):
        filename = f"{PERFORMANCES_FOLDER}/{model_name}_{model_no}_1.csv"
        if os.path.exists(filename):
            performance = pd.read_csv(filename,index_col=0)
            col_name = f"{model_name}_{model_no}_1-baseline"
            single_performances[model_no] = performance[col_name]
    df = pd.DataFrame(pd.DataFrame(single_performances).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose().describe().round(4)
    return df 

In [23]:
model_name = 'lightgbm-gbdt_None_1'
standard_error_models(model_name)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0253,0.0312,-0.0004,0.0706,0.2338,0.8104,0.1100
std,0.0006,0.0006,0.0559,0.0754,0.0296,0.0278,0.0153
min,0.0245,0.0304,-0.1290,-0.0238,0.1975,0.7580,0.0889
25%,0.0249,0.0309,-0.0202,0.0308,0.2106,0.7938,0.0980
50%,0.0252,0.0312,0.0020,0.0558,0.2296,0.8107,0.1104
75%,0.0256,0.0313,0.0429,0.0997,0.2581,0.8342,0.1212
max,0.0263,0.0324,0.0626,0.2447,0.2789,0.8467,0.1327


In [24]:
model_name = 'lightgbm-dart_None_1'
standard_error_models(model_name)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0254,0.0266,-0.0077,0.0344,0.1567,0.9593,0.1639
std,0.0006,0.0007,0.0644,0.1187,0.0158,0.0365,0.0175
min,0.0242,0.0256,-0.1172,-0.1112,0.1405,0.8989,0.1280
25%,0.0251,0.0259,-0.0393,-0.0622,0.1493,0.9317,0.1577
50%,0.0254,0.0266,-0.0239,0.0224,0.1516,0.9722,0.1680
75%,0.0259,0.0269,0.0463,0.1286,0.1614,0.9785,0.1720
max,0.0265,0.0277,0.0916,0.2219,0.1953,1.0119,0.1886


In [25]:
model_name = 'Numerai-MLP_None_1'
standard_error_models(model_name)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0233,0.0271,0.1342,-0.1450,0.1643,0.8600,0.1446
std,0.0009,0.0011,0.0601,0.0705,0.0248,0.0365,0.0219
min,0.0215,0.0247,0.0536,-0.2544,0.1326,0.8099,0.1012
25%,0.0228,0.0266,0.1002,-0.1797,0.1486,0.8418,0.1322
50%,0.0234,0.0272,0.1233,-0.1557,0.1582,0.8557,0.1489
75%,0.0238,0.0278,0.1468,-0.1038,0.1765,0.8732,0.1571
max,0.0245,0.0286,0.2469,-0.0235,0.2124,0.9250,0.1757


### Skip Intial Trees 

In [26]:
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_robustness/trees"

In [27]:
## Baseline Models 
model_name = 'lightgbm-gbdt_None_1_2000_10'
tree_performances = dict()
for trees in [0,100,250,500]:
    filename = f"{PERFORMANCES_FOLDER}/{model_name}_trees_{trees}.csv"
    if os.path.exists(filename):
        tree_performance = pd.read_csv(filename,index_col=0)
        col_name = f"{model_name}-baseline"
        tree_performances[trees] = tree_performance[col_name]

In [28]:
pd.DataFrame(pd.DataFrame(tree_performances).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
0,0.0262,0.0321,0.0033,0.0714,0.2378,0.8140,0.1102
100,0.0265,0.0291,-0.0769,-0.0034,0.1835,0.9106,0.1444
250,0.0253,0.0259,-0.0397,0.1873,0.1490,0.9769,0.1698
500,0.0253,0.0259,-0.0399,0.1856,0.1490,0.9765,0.1698


In [29]:
model_name = 'lightgbm-gbdt_None_1_2000_10'
tree_performances = dict()
for trees in [0,100,250,500]:
    filename = f"{PERFORMANCES_FOLDER}/{model_name}_trees_{trees}.csv"
    if os.path.exists(filename):
        tree_performance = pd.read_csv(filename,index_col=0)
        col_name = f"{model_name}-optimizer"
        tree_performances[trees] = tree_performance[col_name]

In [30]:
pd.DataFrame(pd.DataFrame(tree_performances).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
0,0.0204,0.0211,0.0781,0.1959,0.1998,0.9665,0.1021
100,0.0206,0.0200,0.0531,0.2075,0.1912,1.0293,0.1077
250,0.0194,0.0188,-0.0273,0.4256,0.2058,1.0307,0.0943
500,0.0193,0.0188,-0.0291,0.4253,0.2063,1.0301,0.0936


In [31]:
## Baseline Models 
model_name = 'lightgbm-dart_None_1_2000_10'
tree_performances = dict()
for trees in [0,100,250,500]:
    filename = f"{PERFORMANCES_FOLDER}/{model_name}_trees_{trees}.csv"
    if os.path.exists(filename):
        tree_performance = pd.read_csv(filename,index_col=0)
        col_name = f"{model_name}-baseline"
        tree_performances[trees] = tree_performance[col_name]

In [32]:
pd.DataFrame(pd.DataFrame(tree_performances).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
0,0.0278,0.0284,-0.0012,0.0398,0.1622,0.9791,0.1714
100,0.0272,0.0264,-0.0344,0.0012,0.1384,1.0293,0.1965
250,0.0264,0.0255,-0.0490,0.0231,0.1299,1.0336,0.2032
500,0.0249,0.0238,-0.0370,-0.0040,0.1166,1.0459,0.2136


In [33]:
## Baseline Models 
model_name = 'lightgbm-dart_None_1_2000_10'
tree_performances = dict()
for trees in [0,100,250,500]:
    filename = f"{PERFORMANCES_FOLDER}/{model_name}_trees_{trees}.csv"
    if os.path.exists(filename):
        tree_performance = pd.read_csv(filename,index_col=0)
        col_name = f"{model_name}-optimizer"
        tree_performances[trees] = tree_performance[col_name]

In [34]:
pd.DataFrame(pd.DataFrame(tree_performances).loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict()).transpose()

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar
0,0.0215,0.0182,-0.0933,0.0753,0.1153,1.1806,0.1865
100,0.0208,0.0174,-0.1566,0.0446,0.1079,1.1998,0.1928
250,0.0200,0.0168,-0.2398,0.1252,0.1103,1.1918,0.1813
500,0.0183,0.0156,-0.3058,0.0908,0.1044,1.1748,0.1753


### Random Feature Neutralisation 

Calculate Quantile Stats from models with random feature neutralisaion 

In [35]:
PERFORMANCES_FOLDER = "../numerai_models_performances_v4_robustness/random"


In [36]:
def check_rfn(model_name='lightgbm-dart_None_1_2000_10'):
    files = glob.glob(f"{PERFORMANCES_FOLDER}/{model_name}_*.csv")
    all_random_fnc = list()
    for f in files:
        df = pd.read_csv(f,index_col=0)
        all_random_fnc.append(df)
    all_random_fnc = pd.concat(all_random_fnc,axis=1).dropna()
    quantiles = pd.DataFrame(all_random_fnc.loc['2015-05-15':'2022-09-23'].apply(strategy_metrics).to_dict())\
    .transpose().quantile(q=[0.05,0.1,0.5,0.9,0.95])
    result = switching_dfn(all_random_fnc.loc['2015-05-15':'2022-09-23'],no_tops_list=[10,],random_mode=True)
    return quantiles, result

In [37]:
model_name = 'lightgbm-dart_None_1_2000_10'
quants, leaderboard = check_rfn(model_name)

In [38]:
leaderboard 

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0216,0.0149,0.0871,-0.2290,0.0472,1.4522,0.4576,mean_max,10,52
1,0.0213,0.0147,0.0795,-0.1672,0.0459,1.4474,0.4641,sharpe_max,10,52
2,0.0214,0.0148,0.1039,-0.2212,0.0453,1.4504,0.4724,calmar_max,10,52
3,0.0214,0.0147,0.0681,-0.1540,0.0482,1.4547,0.4440,average,0,52


In [39]:
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0246,0.0180,0.0996,-0.3206,0.0533,1.3654,0.4615,mean_max,1,52
1,0.0234,0.0165,0.0268,-0.3042,0.0533,1.4148,0.4390,sharpe_max,1,52
2,0.0225,0.0171,0.0291,-0.3466,0.0350,1.3122,0.6429,calmar_max,1,52
3,0.0229,0.0160,-0.0200,-0.4117,0.0619,1.4323,0.3700,average,0,52


In [40]:
model_name = 'lightgbm-gbdt_None_1_2000_10'
quants, leaderboard = check_rfn(model_name)

In [41]:
leaderboard

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0208,0.0167,0.1443,-0.0485,0.0641,1.2457,0.3245,mean_max,10,52
1,0.0206,0.0168,0.1953,-0.0649,0.0618,1.2267,0.3333,sharpe_max,10,52
2,0.0203,0.0167,0.1932,-0.0012,0.0740,1.2135,0.2743,calmar_max,10,52
3,0.0203,0.0167,0.2196,-0.0547,0.0664,1.2140,0.3057,average,0,52


In [42]:
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0228,0.0201,0.1367,0.0667,0.0729,1.1342,0.3128,mean_max,1,52
1,0.0224,0.0187,0.0332,-0.0528,0.0729,1.1966,0.3073,sharpe_max,1,52
2,0.0216,0.0195,0.1353,-0.0081,0.0508,1.1102,0.4252,calmar_max,1,52
3,0.0216,0.0177,0.0747,-0.3600,0.0710,1.2165,0.3042,average,0,52


In [43]:
model_name = 'Numerai-MLP_None_1_2000_10'
quants, leaderboard = check_rfn(model_name)

In [44]:
leaderboard

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0179,0.0165,0.1481,0.0524,0.0790,1.0842,0.2266,mean_max,10,52
1,0.0177,0.0164,0.1482,0.0810,0.0762,1.0751,0.2323,sharpe_max,10,52
2,0.0175,0.0167,0.1532,0.0828,0.0825,1.0511,0.2121,calmar_max,10,52
3,0.0176,0.0165,0.1706,0.1417,0.0831,1.0658,0.2118,average,0,52


In [45]:
df = pd.read_csv(f'../numerai_models_performances_v4_thesis/{model_name}.csv',index_col=0)
switching_dfn(df)

,mean,volatility,skew,kurtosis,max_drawdown,sharpe,calmar,method,no_tops,lookback
0,0.0212,0.0191,0.3312,0.0059,0.0878,1.1124,0.2415,mean_max,1,52
1,0.0207,0.0186,0.0690,-0.0971,0.0878,1.1110,0.2358,sharpe_max,1,52
2,0.0187,0.0201,0.0416,0.2272,0.1973,0.9309,0.0948,calmar_max,1,52
3,0.0195,0.0175,0.1543,-0.1751,0.0918,1.1149,0.2124,average,0,52
